<a href="https://colab.research.google.com/github/celinezyyy/Data-Mining/blob/master/22004878_Individual_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import necessary libraries**

In [ ]:
!pip install featuretools

import featuretools as ft
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 16.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
from google.colab import files

drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# **Create DataFrames for Each Entity**

In [ ]:
# Load the Excel file
file_path = "/content/drive/MyDrive/Y3S1/WIE3007/Individual Assignment/E-commerce data.xlsx"

# Load each sheet into a separate DataFrame
customers_df = pd.read_excel(file_path, sheet_name="Customers")
products_df = pd.read_excel(file_path, sheet_name="Products")
orders_df = pd.read_excel(file_path, sheet_name="Orders")
order_details_df = pd.read_excel(file_path, sheet_name="OrderDetails")

In [ ]:
customers_df.head()

,CustomerID,Name,Email,SignupDate
0,101,John Doe,john.doe@gmail.com,2023-01-10
1,102,Jane Smith,jane.smith@gmail.com,2023-01-15
2,103,Mike Jordan,mike.jordan@gmail.com,2023-01-20
3,104,Celine Ong,celine@gmail.com,2023-01-21
4,105,Louis Koh,louis@gmail.com,2023-01-22


In [ ]:
products_df.head()

,ProductID,Name,Category,Price
0,201,Laptop,Electronics,1000
1,202,Tablet,Electronics,500
2,203,Smartphone,Electronics,800
3,204,Headphones,Electronics,150
4,205,Smartwatch,Electronics,250


In [ ]:
orders_df.head()

,OrderID,CustomerID,OrderDate,ShipDate
0,301,101,2023-02-01,2023-02-03
1,302,102,2023-02-05,2023-02-07
2,303,103,2023-02-10,2023-02-12
3,304,103,2023-02-15,2023-02-18
4,305,104,2023-02-15,2023-02-18


In [ ]:
order_details_df.head()

,OrderID,ProductID,Quantity,Discount
0,301,201,1,0.00
1,302,202,2,0.10
2,303,203,1,0.00
3,304,201,2,0.20
4,304,204,1,0.05


# **Create EntitySet and Add Entities**

In [ ]:
# Create an empty EntitySet for the e-commerce data
entity_set = ft.EntitySet(id='ecommerce_set')

# Add Customers entity to the EntitySet
customers_entity = entity_set.add_dataframe(
    dataframe_name='Customers',
    dataframe=customers_df,
    index='CustomerID',
    time_index='SignupDate'
)

# Add Products entity to the EntitySet
products_entity = entity_set.add_dataframe(
    dataframe_name='Products',
    dataframe=products_df,
    index='ProductID'
)

# Add Orders entity to the EntitySet
orders_entity = entity_set.add_dataframe(
    dataframe_name='Orders',
    dataframe=orders_df,
    index='OrderID',
    time_index='OrderDate'
)

# Add OrderDetails entity to the EntitySet
order_details_entity = entity_set.add_dataframe(
    dataframe_name='OrdersDetails',
    dataframe=order_details_df,
    index='OrderID,ProductID'
)

/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

# **Define Relationships**

In [ ]:
# Define relationships (foreign keys linking tables)
relationship_1 = entity_set.add_relationship(
    parent_dataframe_name='Customers',
    parent_column_name='CustomerID',
    child_dataframe_name='Orders',
    child_column_name='CustomerID'
)

relationship_2 = entity_set.add_relationship(
    parent_dataframe_name='Products',
    parent_column_name='ProductID',
    child_dataframe_name='OrdersDetails',
    child_column_name='ProductID'
)

relationship_3 = entity_set.add_relationship(
    parent_dataframe_name='Orders',
    parent_column_name='OrderID',
    child_dataframe_name='OrdersDetails',
    child_column_name='OrderID'
)

In [ ]:
print(entity_set)

Entityset: ecommerce_set
  DataFrames:
    Customers [Rows: 40, Columns: 4]
    Products [Rows: 15, Columns: 4]
    Orders [Rows: 39, Columns: 4]
    OrdersDetails [Rows: 41, Columns: 5]
  Relationships:
    Orders.CustomerID -> Customers.CustomerID
    OrdersDetails.ProductID -> Products.ProductID
    OrdersDetails.OrderID -> Orders.OrderID


In [ ]:
# SET PANDAS DISPLAY OPTIONS TO SHOW ALL COLUMNS AND ROWS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# **Automated Feature Generation Using Featuretools**

In [ ]:
# Automatically generate features using Orders dataframe
feature_matrix_orders, feature_defs = ft.dfs(entityset=entity_set, target_dataframe_name="Orders", verbose=True, max_depth=2)

# View generated features
feature_matrix_orders.head()

Built 48 features
Elapsed: 00:00 | Progress:  36%|███▌      

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x7c526ebe5120> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x7c526ebe4820> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x7c526ebe4700> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will b

Elapsed: 00:00 | Progress:  95%|█████████▌

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x7c526ebe4700> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x7c526ebe5120> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x7c526ebe40d0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will b

Elapsed: 00:00 | Progress: 100%|██████████


,CustomerID,COUNT(OrdersDetails),MAX(OrdersDetails.Discount),MAX(OrdersDetails.Quantity),MEAN(OrdersDetails.Discount),MEAN(OrdersDetails.Quantity),MIN(OrdersDetails.Discount),MIN(OrdersDetails.Quantity),SKEW(OrdersDetails.Discount),SKEW(OrdersDetails.Quantity),STD(OrdersDetails.Discount),STD(OrdersDetails.Quantity),SUM(OrdersDetails.Discount),SUM(OrdersDetails.Quantity),DAY(OrderDate),DAY(ShipDate),MONTH(OrderDate),MONTH(ShipDate),WEEKDAY(OrderDate),WEEKDAY(ShipDate),YEAR(OrderDate),YEAR(ShipDate),MAX(OrdersDetails.Products.Price),MEAN(OrdersDetails.Products.Price),MIN(OrdersDetails.Products.Price),MODE(OrdersDetails.Products.Category),NUM_UNIQUE(OrdersDetails.Products.Category),SKEW(OrdersDetails.Products.Price),STD(OrdersDetails.Products.Price),SUM(OrdersDetails.Products.Price),Customers.COUNT(Orders),Customers.COUNT(OrdersDetails),Customers.MAX(OrdersDetails.Discount),Customers.MAX(OrdersDetails.Quantity),Customers.MEAN(OrdersDetails.Discount),Customers.MEAN(OrdersDetails.Quantity),Customers.MIN(OrdersDetails.Discount),Customers.MIN(OrdersDetails.Quantity),Customers.SKEW(OrdersDetails.Discount),Customers.SKEW(OrdersDetails.Quantity),Customers.STD(OrdersDetails.Discount),Customers.STD(OrdersDetails.Quantity),Customers.SUM(OrdersDetails.Discount),Customers.SUM(OrdersDetails.Quantity),Customers.DAY(SignupDate),Customers.MONTH(SignupDate),Customers.WEEKDAY(SignupDate),Customers.YEAR(SignupDate)
OrderID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
301,101,1,0.00,1.0,0.000,1.0,0.00,1.0,NaN,NaN,NaN,NaN,0.00,1.0,1,3,2,2,2,4,2023,2023,1000.0,1000.0,1000.0,Electronics,1,NaN,NaN,1000.0,2,2,0.10,2.0,0.0500,1.500000,0.00,1.0,NaN,NaN,0.070711,0.707107,0.10,3.0,10,1,1,2023
302,102,1,0.10,2.0,0.100,2.0,0.10,2.0,NaN,NaN,NaN,NaN,0.10,2.0,5,7,2,2,6,1,2023,2023,500.0,500.0,500.0,Electronics,1,NaN,NaN,500.0,3,3,0.15,2.0,0.1000,1.666667,0.05,1.0,0.000000,-1.732051,0.050000,0.577350,0.30,5.0,15,1,6,2023
303,103,1,0.00,1.0,0.000,1.0,0.00,1.0,NaN,NaN,NaN,NaN,0.00,1.0,10,12,2,2,4,6,2023,2023,800.0,800.0,800.0,Electronics,1,NaN,NaN,800.0,3,4,0.20,2.0,0.0625,1.250000,0.00,1.0,1.658524,2.000000,0.094648,0.500000,0.25,5.0,20,1,4,2023
304,103,2,0.20,2.0,0.125,1.5,0.05,1.0,NaN,NaN,0.106066,0.707107,0.25,3.0,15,18,2,2,2,5,2023,2023,1000.0,575.0,150.0,Electronics,1,NaN,601.040764,1150.0,3,4,0.20,2.0,0.0625,1.250000,0.00,1.0,1.658524,2.000000,0.094648,0.500000,0.25,5.0,20,1,4,2023
305,104,1,0.05,1.0,0.050,1.0,0.05,1.0,NaN,NaN,NaN,NaN,0.05,1.0,15,18,2,2,2,5,2023,2023,1200.0,1200.0,1200.0,Electronics,1,NaN,NaN,1200.0,4,4,0.10,2.0,0.0625,1.500000,0.00,1.0,-0.854563,0.000000,0.047871,0.577350,0.25,6.0,21,1,5,2023


In [ ]:
feature_matrix_orders.columns

Index(['CustomerID', 'COUNT(OrdersDetails)', 'MAX(OrdersDetails.Discount)',
       'MAX(OrdersDetails.Quantity)', 'MEAN(OrdersDetails.Discount)',
       'MEAN(OrdersDetails.Quantity)', 'MIN(OrdersDetails.Discount)',
       'MIN(OrdersDetails.Quantity)', 'SKEW(OrdersDetails.Discount)',
       'SKEW(OrdersDetails.Quantity)', 'STD(OrdersDetails.Discount)',
       'STD(OrdersDetails.Quantity)', 'SUM(OrdersDetails.Discount)',
       'SUM(OrdersDetails.Quantity)', 'DAY(OrderDate)', 'DAY(ShipDate)',
       'MONTH(OrderDate)', 'MONTH(ShipDate)', 'WEEKDAY(OrderDate)',
       'WEEKDAY(ShipDate)', 'YEAR(OrderDate)', 'YEAR(ShipDate)',
       'MAX(OrdersDetails.Products.Price)',
       'MEAN(OrdersDetails.Products.Price)',
       'MIN(OrdersDetails.Products.Price)',
       'MODE(OrdersDetails.Products.Category)',
       'NUM_UNIQUE(OrdersDetails.Products.Category)',
       'SKEW(OrdersDetails.Products.Price)',
       'STD(OrdersDetails.Products.Price)',
       'SUM(OrdersDetails.Products.Price)', '

In [ ]:
feature_matrix_orders.to_csv('feature_matrix.csv', index = False)

In [ ]:
for column in feature_matrix_orders.columns:
    print(column)

CustomerID
COUNT(OrdersDetails)
MAX(OrdersDetails.Discount)
MAX(OrdersDetails.Quantity)
MEAN(OrdersDetails.Discount)
MEAN(OrdersDetails.Quantity)
MIN(OrdersDetails.Discount)
MIN(OrdersDetails.Quantity)
SKEW(OrdersDetails.Discount)
SKEW(OrdersDetails.Quantity)
STD(OrdersDetails.Discount)
STD(OrdersDetails.Quantity)
SUM(OrdersDetails.Discount)
SUM(OrdersDetails.Quantity)
DAY(OrderDate)
DAY(ShipDate)
MONTH(OrderDate)
MONTH(ShipDate)
WEEKDAY(OrderDate)
WEEKDAY(ShipDate)
YEAR(OrderDate)
YEAR(ShipDate)
MAX(OrdersDetails.Products.Price)
MEAN(OrdersDetails.Products.Price)
MIN(OrdersDetails.Products.Price)
MODE(OrdersDetails.Products.Category)
NUM_UNIQUE(OrdersDetails.Products.Category)
SKEW(OrdersDetails.Products.Price)
STD(OrdersDetails.Products.Price)
SUM(OrdersDetails.Products.Price)
Customers.COUNT(Orders)
Customers.COUNT(OrdersDetails)
Customers.MAX(OrdersDetails.Discount)
Customers.MAX(OrdersDetails.Quantity)
Customers.MEAN(OrdersDetails.Discount)
Customers.MEAN(OrdersDetails.Quantity)
C

In [ ]:
from google.colab import files
files.download('feature_matrix.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Finding Insights And Display in Dashboard**

In [ ]:
# !pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataframe = pd.read_csv('feature_matrix.csv')

dataframe.head()

,CustomerID,COUNT(OrdersDetails),MAX(OrdersDetails.Discount),MAX(OrdersDetails.Quantity),MEAN(OrdersDetails.Discount),MEAN(OrdersDetails.Quantity),MIN(OrdersDetails.Discount),MIN(OrdersDetails.Quantity),SKEW(OrdersDetails.Discount),SKEW(OrdersDetails.Quantity),STD(OrdersDetails.Discount),STD(OrdersDetails.Quantity),SUM(OrdersDetails.Discount),SUM(OrdersDetails.Quantity),DAY(OrderDate),DAY(ShipDate),MONTH(OrderDate),MONTH(ShipDate),WEEKDAY(OrderDate),WEEKDAY(ShipDate),YEAR(OrderDate),YEAR(ShipDate),MAX(OrdersDetails.Products.Price),MEAN(OrdersDetails.Products.Price),MIN(OrdersDetails.Products.Price),MODE(OrdersDetails.Products.Category),NUM_UNIQUE(OrdersDetails.Products.Category),SKEW(OrdersDetails.Products.Price),STD(OrdersDetails.Products.Price),SUM(OrdersDetails.Products.Price),Customers.COUNT(Orders),Customers.COUNT(OrdersDetails),Customers.MAX(OrdersDetails.Discount),Customers.MAX(OrdersDetails.Quantity),Customers.MEAN(OrdersDetails.Discount),Customers.MEAN(OrdersDetails.Quantity),Customers.MIN(OrdersDetails.Discount),Customers.MIN(OrdersDetails.Quantity),Customers.SKEW(OrdersDetails.Discount),Customers.SKEW(OrdersDetails.Quantity),Customers.STD(OrdersDetails.Discount),Customers.STD(OrdersDetails.Quantity),Customers.SUM(OrdersDetails.Discount),Customers.SUM(OrdersDetails.Quantity),Customers.DAY(SignupDate),Customers.MONTH(SignupDate),Customers.WEEKDAY(SignupDate),Customers.YEAR(SignupDate)
0,101,1,0.00,1.0,0.000,1.0,0.00,1.0,NaN,NaN,NaN,NaN,0.00,1.0,1,3,2,2,2,4,2023,2023,1000.0,1000.0,1000.0,Electronics,1,NaN,NaN,1000.0,2,2,0.10,2.0,0.0500,1.500000,0.00,1.0,NaN,NaN,0.070711,0.707107,0.10,3.0,10,1,1,2023
1,102,1,0.10,2.0,0.100,2.0,0.10,2.0,NaN,NaN,NaN,NaN,0.10,2.0,5,7,2,2,6,1,2023,2023,500.0,500.0,500.0,Electronics,1,NaN,NaN,500.0,3,3,0.15,2.0,0.1000,1.666667,0.05,1.0,0.000000,-1.732051,0.050000,0.577350,0.30,5.0,15,1,6,2023
2,103,1,0.00,1.0,0.000,1.0,0.00,1.0,NaN,NaN,NaN,NaN,0.00,1.0,10,12,2,2,4,6,2023,2023,800.0,800.0,800.0,Electronics,1,NaN,NaN,800.0,3,4,0.20,2.0,0.0625,1.250000,0.00,1.0,1.658524,2.000000,0.094648,0.500000,0.25,5.0,20,1,4,2023
3,103,2,0.20,2.0,0.125,1.5,0.05,1.0,NaN,NaN,0.106066,0.707107,0.25,3.0,15,18,2,2,2,5,2023,2023,1000.0,575.0,150.0,Electronics,1,NaN,601.040764,1150.0,3,4,0.20,2.0,0.0625,1.250000,0.00,1.0,1.658524,2.000000,0.094648,0.500000,0.25,5.0,20,1,4,2023
4,104,1,0.05,1.0,0.050,1.0,0.05,1.0,NaN,NaN,NaN,NaN,0.05,1.0,15,18,2,2,2,5,2023,2023,1200.0,1200.0,1200.0,Electronics,1,NaN,NaN,1200.0,4,4,0.10,2.0,0.0625,1.500000,0.00,1.0,-0.854563,0.000000,0.047871,0.577350,0.25,6.0,21,1,5,2023
